# Source

https://www.kaggle.com/datasets/nikdavis/steam-store-games

# Introduction

Steam is a popular digital distribution platform developed by Valve Corporation, which offers a vast library of video games, software, and other multimedia content. It provides both a desktop application and a web platform, allowing users to purchase, download, and play their favorite games with ease. The platform is known for its vast selection, user-friendly interface, and frequent sales, making it a go-to choice for gamers worldwide.

Recommendation systems are intelligent algorithms designed to provide personalized suggestions to users based on their preferences, behavior, or other relevant factors. These systems analyze data patterns to predict and recommend items that are likely to be of interest, enhancing user experience and engagement with the platform.

Steam uses a recommendation system to suggest games that users might be interested in based on their play history, friends' activity, and other factors. However, not all users log in when browsing the Steam store through a web browser, resulting in a lack of access to personalized data such as their game library and play history. This limitation poses a challenge to provide relevant game recommendations to users who are not logged in.

## Business question

The business question addressed in this project is: <br>
How can a recommendation system for the Steam platform that relies solely on game similarities be built, without requiring users to log in or access their personal library and play history? 

This approach will enable Steam to provide valuable suggestions to users who browse the platform without logging in, potentially increasing engagement and sales.

## Dataset Description

The following three datasets from Kaggle have been used in this project to build the game similarity-based recommendation system:

### 1. steam.csv
This dataset contains general information about the games available on Steam, including their title, release date, developer, publisher, genres, and other relevant details. The dataset has the following columns:

- **name**: The title of the game.
- **release_date**: The date the game was released.
- **english**: A binary indicator of whether the game supports the English language (1) or not (0).
- **developer**: The developer responsible for creating the game.
- **publisher**: The company responsible for publishing the game.
- **platforms**: The platforms the game is available on (e.g., Windows, Mac, Linux).
- **required_age**: The minimum age requirement to play the game, according to ESRB ratings or similar guidelines.
- **categories**: A list of categories that describe the game's features (e.g., Single-player, Multi-player, Co-op).
- **genres**: The genre(s) the game belongs to.
- **steamspy_tags**: The top three tags associated with the game, according to SteamSpy data.
- **achievements**: The number of achievements available in the game.
- **positive_ratings**: The number of positive ratings the game has received from users.
- **negative_ratings**: The number of negative ratings the game has received from users.
- **average_playtime**: The average playtime of the game, in minutes, among users who have played the game.
- **median_playtime**: The median playtime of the game, in minutes, among users who have played the game.
- **owners**: An estimated range of the total number of users who own the game (e.g., "20000-50000").
- **price**: The current price of the game in USD.

This dataset is essential for the project as it provides the basic information about the games and allows for identifying the key characteristics of each game to be used in the similarity calculations.


### 2. steam_description_data.csv
This dataset provides additional information about the games in the form of descriptive texts, including a short description, detailed description, and other related texts. The columns in this dataset are:

- **steam_appid**: Unique identifier for each game, matching the appid in steam.csv.
- **detailed_description**: A more comprehensive description of the game, often including gameplay features and storyline.
- **about_the_game**: A brief overview of the game, highlighting its main features and selling points.
- **short_description**: A concise summary of the game.

The descriptive texts in this dataset are important for understanding the content and context of each game. By analyzing these texts, relevant features can be extracted to determine the similarities between games.

### 3. steamspy_tag_data.csv
This dataset contains user-generated tags for each game, which help categorize and describe the games in more detail. The columns in this dataset are:

- **appid**: Unique identifier for each game, matching the appid in steam.csv.
- **[tag_name]**: Each column represents a different tag, and the value in each cell indicates the number of times that tag has been applied to the corresponding game by users.

The user-generated tags in this dataset provide valuable insights into the popular features and characteristics of each game, as perceived by the community. Analyzing these tags helps us to determine the similarities between games based on the preferences and opinions of Steam users.


By combining the information from these three datasets, a comprehensive understanding of each game's features, content, and user perception can be created. This data will enable us to build an effective similarity-based recommendation system for users who browse the Steam platform without logging in.


# Importing Libraries and Loading Datasets

In this step, I imported the necessary libraries and load the three datasets into separate DataFrame objects. 

I also set the index of each DataFrame to the unique game identifier (`appid` or `steam_appid`) for easier data manipulation. These values are key and foreign keys of the database.


In [2]:
import pandas as pd
import numpy as np
import spacy
from langdetect import detect
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import pickle
from time import time

# Load Spacy's English model, using the medium-sized model for a balance between accuracy and processing time
nlp = spacy.load('en_core_web_md')

In [3]:
df = pd.read_csv('steam.csv')
df.set_index('appid', inplace=True)

tag_df = pd.read_csv('steamspy_tag_data.csv')
tag_df.set_index('appid', inplace=True)

des_df = pd.read_csv('steam_description_data.csv')
des_df.set_index('steam_appid', inplace=True)


pd.set_option('display.max_columns', 400)



# Data Exploration and Preprocessing: main dataframe

Before building the recommendation system, I explored the `steam` DataFrame and preprocess the data to ensure its quality and relevance.

### Findings
- 77% of the dataset has implicit missing values for the `average_playtime` column, making it unsuitable for use in the recommendation system.
- 511 games are not in English. These will be removed to focus on English-language games.

### Preprocessing Steps
1. Drop the non-English games from the `steam` DataFrame.
2. Remove unnecessary columns that are not relevant for the recommendation system: ['platforms', 'average_playtime', 'median_playtime', 'price', 'required_age', 'achievements', 'steamspy_tags', 'english'].


In [4]:
print(df.shape)
df.head()

(27075, 17)


,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
appid,,,,,,,,,,,,,,,,,
10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [5]:
len(df[(df['average_playtime']==0)])/len(df)

0.7721144967682364

In [6]:
print(df[df['english']==0].shape[0])
df = df[df['english']!=0]

511


In [7]:
df.drop(columns=['platforms', 'average_playtime', 'median_playtime', 'price',
                 'required_age', 'achievements', 'steamspy_tags', 'english'],
                 inplace=True)
df.head()

,name,release_date,developer,publisher,categories,genres,positive_ratings,negative_ratings,owners
appid,,,,,,,,,
10,Counter-Strike,2000-11-01,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,124534,3339,10000000-20000000
20,Team Fortress Classic,1999-04-01,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,3318,633,5000000-10000000
30,Day of Defeat,2003-05-01,Valve,Valve,Multi-player;Valve Anti-Cheat enabled,Action,3416,398,5000000-10000000
40,Deathmatch Classic,2001-06-01,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,1273,267,5000000-10000000
50,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,Single-player;Multi-player;Valve Anti-Cheat en...,Action,5250,288,5000000-10000000


# Data Exploration and Preprocessing: Description DataFrame

In this step, I explored the `des_df` DataFrame and preprocess the data to ensure its quality and relevance for building the recommendation system.

### Data Exploration Findings
- The 'about_the_game' column provides a good balance between the amount of text context and information needed for creating meaningful embeddings using Spacy's document vectorizer.
- Many HTML code sections were found in the text data. These may negatively affect the document vectorization process.

### Preprocessing Steps
1. Select the 'about_the_game' column for further processing, as it provides sufficient context and information for the recommendation system.
2. Replace HTML code sections with a whitespace to prevent joining words together and to ensure that Spacy's pipeline can handle the text data more effectively.
3. Focus on games with English descriptions, as they form the majority of the dataset and allow us to use Spacy's pre-trained English model.



In [8]:
des_df.sample(5)

,detailed_description,about_the_game,short_description
steam_appid,,,
744550,Top-down tactical shooter with stealth element...,Top-down tactical shooter with stealth element...,Tactical Operations is a subsystem of investig...
875240,Many of us live in a noisy urban environment. ...,Many of us live in a noisy urban environment. ...,An artistic bonsai tree simulation game that f...
758100,"<h2 class=""bb_tag""><strong>About E-Startup</st...","<h2 class=""bb_tag""><strong>About E-Startup</st...",E-Startup is a sandbox business simulation gam...
503480,"Run, jump and slash your way through an epic, ...","Run, jump and slash your way through an epic, ...",Mahluk is a hack and slash platformer game wit...
995000,You Play as Erica who follows the adventurous ...,You Play as Erica who follows the adventurous ...,"Explore, find clues, solve puzzles and make yo..."


In [9]:
des_df = des_df[['about_the_game']].copy()
des_df

,about_the_game
steam_appid,
10,Play the world's number 1 online action game. ...
20,One of the most popular online action games of...
30,Enlist in an intense brand of Axis vs. Allied ...
40,Enjoy fast-paced multiplayer gaming with Death...
50,Return to the Black Mesa Research Facility as ...
...,...
1065230,"<img src=""https://steamcdn-a.akamaihd.net/stea..."
1065570,Have you ever been so lonely that no one but y...
1065650,<strong>Super Star Blast </strong>is a space b...


In [10]:
# Print five random 'about_the_game' contents
def show_rnd_strings(des_df):
    rn_indexes = des_df['about_the_game'].sample(5).index
    for i in rn_indexes:
        print(des_df['about_the_game'].loc[i])
        print()
        
# show_rnd_strings(des_df)

In [11]:
# Remove html code
pattern = '<.*?>'
des_df['about_the_game'] = des_df['about_the_game'].replace(pattern, ' ', regex=True)

# show_rnd_strings(des_df) 

In [12]:
# remove non english content
def is_english(text):
    try:
        lang = detect(text)
        if lang == 'en':
            return True
        else:
            return False
    except:
        return False

# Filter rows that are in English
des_df = des_df[des_df['about_the_game'].apply(is_english)]

## Data Exploration and Preprocessing: Tags DataFrame

In this step, I explored the `tag_df` DataFrame and preprocess the data to ensure its quality and relevance for building the recommendation system.

### Data Exploration Findings
- There are 575 games with no tags. These games will be removed from the dataset as they do not provide any useful information for the recommendation system.

### Preprocessing Steps
- Drop the games with no tags from the `tag_df` DataFrame.



In [13]:
print(len(tag_df[tag_df.sum(axis=1)==0]), 'games have no tags')
tag_df = tag_df[tag_df.sum(axis=1) != 0]

575 games have no tags


# Selecting Games with Complete Information

To ensure the recommendation system has sufficient data for each game, I selected only the games that have complete information across all three DataFrames.

#### Selection
I performed an inner join on the three DataFrames to select only games that have all information available.



In [14]:
print('df rows:', df.shape[0])
print('tag_df rows:', tag_df.shape[0])
print('des_df rows:', des_df.shape[0])

df rows: 26564
tag_df rows: 28447
des_df rows: 27085


In [15]:
df = df.join(des_df, how='inner')
df = df.join(tag_df, how='inner')
print(df.shape)

(26507, 381)


# Creating Data Subsets

To facilitate further data processing and feature engineering, separate DataFrames are created for each relevant aspect of the games.

### Data Subsets
1. **name_df**: Contains the names of the games.
2. **gen_df**: Contains the genres of the games.
3. **cat_df**: Contains the categories of the games.
4. **tag_df**: Contains the user-generated tags for the games.
5. **rating_df**: Contains the positive and negative ratings, as well as the number of owners for the games.
6. **date_df**: Contains the release dates of the games.
7. **dev_df**: Contains the developers of the games.
8. **pub_df**: Contains the publishers of the games.
9. **des_df**: Contains the 'about_the_game' descriptions of the games.
10. **scores_df**: An empty DataFrame to store the final scores used for the recommendation system.



In [16]:
name_df = df[['name']].copy()

gen_df = df[['genres']].copy()
cat_df = df[['categories']].copy()
tag_df = tag_df.loc[:, '1980s':].copy()
rating_df = df[['positive_ratings', 'negative_ratings', 'owners']].copy()
date_df = df[['release_date']].copy()
dev_df = df[['developer']].copy()
pub_df = df[['publisher']].copy()
des_df = df[['about_the_game']].copy()

scores_df = df[[]].copy()


## Transforming Category and Genre Data

Category and genre data are stored as single strings with multiple elements separated by semicolons. To facilitate the use of Jaccard similarity later on, these strings are transformed into lists.

### Data Transformation Steps
1. Convert the 'categories' and 'genres' columns into lists by splitting the strings using the semicolon delimiter.
2. Retain only the relevant categories: 'Co-op', 'Local Co-op', 'Local Multi-Player', 'MMO', 'Multi-player', 'Online Co-op', 'Online Multi-Player', 'Shared/Split Screen', and 'Single-player'.
3. Remove redundant information: Use the 'MMO' category to represent all massively multiplayer online games, as all games with the 'MMO' tag have the 'Massively Multiplayer' category.



In [17]:
def get_all_list_elements(df, column):
    df_genres = df.explode(column)
    unique_genres = df_genres[column].unique()
    # Print the unique genres
    return (list(unique_genres))

In [18]:
cat_df['cat_list'] = cat_df['categories'].apply(lambda x: x.split(';'))
cat_df.drop(columns=['categories'], inplace=True)
print(get_all_list_elements(cat_df, 'cat_list'))

cat_df.sample(5)

['Multi-player', 'Online Multi-Player', 'Local Multi-Player', 'Valve Anti-Cheat enabled', 'Single-player', 'Steam Cloud', 'Steam Achievements', 'Steam Trading Cards', 'Captions available', 'Partial Controller Support', 'Includes Source SDK', 'Cross-Platform Multiplayer', 'Stats', 'Commentary available', 'Includes level editor', 'Steam Workshop', 'In-App Purchases', 'Co-op', 'Full controller support', 'Steam Leaderboards', 'SteamVR Collectibles', 'Online Co-op', 'Shared/Split Screen', 'Local Co-op', 'MMO', 'VR Support', 'Mods', 'Mods (require HL2)', 'Steam Turn Notifications']


,cat_list
705210,"[Single-player, Multi-player, Online Multi-Pla..."
701900,"[Single-player, Partial Controller Support]"
240360,"[Single-player, Steam Achievements, Partial Co..."
854740,[Single-player]
532840,"[Single-player, Steam Achievements]"


In [19]:
keep_set = {'Co-op', 'Local Co-op', 'Local Multi-Player', 'MMO', 'Multi-player', 'Online Co-op',
            'Online Multi-Player', 'Shared/Split Screen', 'Single-player'}

cat_df['cat_list'] = cat_df['cat_list'].apply(lambda x: list(set(x).intersection(keep_set)) if len(x)>0 else ['unknown'])
cat_df['cat_list'] = cat_df['cat_list'].apply(lambda x: x if len(x)>0 else ['unknown'])

get_all_list_elements(cat_df, 'cat_list')


['Multi-player',
 'Online Multi-Player',
 'Local Multi-Player',
 'Single-player',
 'Co-op',
 'unknown',
 'Online Co-op',
 'Shared/Split Screen',
 'Local Co-op',
 'MMO']

In [20]:
df[cat_df['cat_list'].apply(lambda x: True if 'unknown' in x else False)].shape[0]

186

In [21]:
gen_df['gen_list'] = gen_df['genres'].apply(lambda x: x.split(';'))
gen_df.drop(columns=['genres'], inplace=True)
print(get_all_list_elements(gen_df, 'gen_list'))


gen_df.sample(5)

['Action', 'Free to Play', 'Strategy', 'Adventure', 'Indie', 'RPG', 'Animation & Modeling', 'Video Production', 'Casual', 'Simulation', 'Racing', 'Violent', 'Massively Multiplayer', 'Nudity', 'Sports', 'Early Access', 'Gore', 'Utilities', 'Design & Illustration', 'Web Publishing', 'Education', 'Software Training', 'Sexual Content', 'Audio Production', 'Game Development', 'Photo Editing', 'Accounting', 'Documentary', 'Tutorial']


,gen_list
827900,"[Action, Casual, Indie]"
951010,"[Action, Adventure, Indie]"
819940,"[Action, Indie, Early Access]"
631990,"[Violent, Adventure, Indie, Strategy]"
681150,"[Action, Adventure, Casual, Indie, Racing, Spo..."


#### Exploring MMO

In [22]:
len(df[cat_df['cat_list'].apply(lambda x: True if 'MMO' in x else False)])#[['name','categories','genres']])


400

In [23]:
len(df[gen_df['gen_list'].apply(lambda x: True if 'Massively Multiplayer' in x else False)][['categories','genres']])


692

In [24]:
# Games that has MMO tag and Massively Multiplayer genre
mmo1 = len(df[(cat_df['cat_list'].apply(lambda x: True if 'MMO' in x else False))&
              (gen_df['gen_list'].apply(lambda x: True if 'Massively Multiplayer' in x else False))])
# Games that has nor MMO tag but Massively Multiplayer genre
mmo2 = len(df[(cat_df['cat_list'].apply(lambda x: False if 'MMO' in x else True))&
              (gen_df['gen_list'].apply(lambda x: True if 'Massively Multiplayer' in x else False))])
mmo1+mmo2

692

In [25]:
gen_df[(gen_df['gen_list'].apply(lambda x: True if 'Massively Multiplayer' in x else False))&
       (gen_df['gen_list'].apply(lambda x: True if len(x)==1 else False))]

,gen_list


In [26]:
i = cat_df[(gen_df['gen_list'].apply(lambda x: True if 'Massively Multiplayer' in x else False)) &
           (cat_df['cat_list'].apply(lambda x: False if 'MMO' in x else True))].index

cat_df.loc[i, 'cat_list'] = np.array([x + ['MMO'] for x in cat_df.loc[i, 'cat_list']], dtype=object)

gen_df['gen_list'] = gen_df['gen_list'].apply(lambda x: [g for g in x if g != 'Massively Multiplayer'])


In [27]:
get_all_list_elements(gen_df, 'gen_list')

['Action',
 'Free to Play',
 'Strategy',
 'Adventure',
 'Indie',
 'RPG',
 'Animation & Modeling',
 'Video Production',
 'Casual',
 'Simulation',
 'Racing',
 'Violent',
 'Nudity',
 'Sports',
 'Early Access',
 'Gore',
 'Utilities',
 'Design & Illustration',
 'Web Publishing',
 'Education',
 'Software Training',
 'Sexual Content',
 'Audio Production',
 'Game Development',
 'Photo Editing',
 'Accounting',
 'Documentary',
 'Tutorial']

## Normalizing Tag Data

To ensure that the tag data are on a consistent scale, the values in the `tag_df` DataFrame are normalized to be between 0 and 1 by dividing each value by the maximum value in its row.


In [28]:
def divide_by_max(row):
    max_val = row.max()
    if max_val != 0:
        return row / max_val
    else:
        return 0
    
tag_df = tag_df.apply(divide_by_max, axis=1)

tag_df.sample(5)

,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,4_player_local,4x,6dof,atv,abstract,action,action_rpg,action_adventure,addictive,adventure,agriculture,aliens,alternate_history,america,animation_&_modeling,anime,arcade,arena_shooter,artificial_intelligence,assassin,asynchronous_multiplayer,atmospheric,audio_production,bmx,base_building,baseball,based_on_a_novel,basketball,batman,battle_royale,beat_em_up,beautiful,benchmark,bikes,blood,board_game,bowling,building,bullet_hell,bullet_time,crpg,capitalism,card_game,cartoon,cartoony,casual,cats,character_action_game,character_customization,chess,choices_matter,choose_your_own_adventure,cinematic,city_builder,class_based,classic,clicker,co_op,co_op_campaign,cold_war,colorful,comedy,comic_book,competitive,conspiracy,controller,conversation,crafting,crime,crowdfunded,cult_classic,cute,cyberpunk,cycling,dark,dark_comedy,dark_fantasy,dark_humor,dating_sim,demons,design_&_illustration,destruction,detective,difficult,dinosaurs,diplomacy,documentary,dog,dragons,drama,driving,dungeon_crawler,dungeons_&_dragons,dynamic_narration,dystopian_,early_access,economy,education,emotional,epic,episodic,experience,experimental,exploration,fmv,fps,faith,family_friendly,fantasy,fast_paced,feature_film,female_protagonist,fighting,first_person,fishing,flight,football,foreign,free_to_play,funny,futuristic,gambling,game_development,gamemaker,games_workshop,gaming,god_game,golf,gore,gothic,grand_strategy,great_soundtrack,grid_based_movement,gun_customization,hack_and_slash,hacking,hand_drawn,hardware,heist,hex_grid,hidden_object,historical,hockey,horror,horses,hunting,illuminati,indie,intentionally_awkward_controls,interactive_fiction,inventory_management,investigation,isometric,jrpg,jet,kickstarter,lego,lara_croft,lemmings,level_editor,linear,local_co_op,local_multiplayer,logic,loot,lore_rich,lovecraftian,mmorpg,moba,magic,management,mars,martial_arts,massively_multiplayer,masterpiece,match_3,mature,mechs,medieval,memes,metroidvania,military,mini_golf,minigames,minimalist,mining,mod,moddable,modern,motocross,motorbike,mouse_only,movie,multiplayer,multiple_endings,music,music_based_procedural_generation,mystery,mystery_dungeon,mythology,nsfw,narration,naval,ninja,noir,nonlinear,nudity,offroad,old_school,on_rails_shooter,online_co_op,open_world,otome,parkour,parody_,party_based_rpg,perma_death,philisophical,photo_editing,physics,pinball,pirates,pixel_graphics,platformer,point_&_click,political,politics,pool,post_apocalyptic,procedural_generation,programming,psychedelic,psychological,psychological_horror,puzzle,puzzle_platformer,pve,pvp,quick_time_events,rpg,rpgmaker,rts,racing,real_time_tactics,real_time,real_time_with_pause,realistic,relaxing,remake,replay_value,resource_management,retro,rhythm,robots,rogue_like,rogue_lite,romance,rome,runner,sailing,sandbox,satire,sci_fi,science,score_attack,sequel,sexual_content,shoot_em_up,shooter,short,side_scroller,silent_protagonist,simulation,singleplayer,skateboarding,skating,skiing,sniper,snow,snowboarding,soccer,software,software_training,sokoban,souls_like,soundtrack,space,space_sim,spectacle_fighter,spelling,split_screen,sports,star_wars,stealth,steam_machine,steampunk,story_rich,strategy,strategy_rpg,stylized,submarine,superhero,supernatural,surreal,survival,survival_horror,swordplay,tactical,tactical_rpg,tanks,team_based,tennis,text_based,third_person,third_person_shooter,thriller,time_attack,time_management,time_manipulation,time_travel,top_down,top_down_shooter,touch_friendly,tower_defense,trackir,trading,trading_card_game,trains,transhumanism,turn_based,turn_based_combat,turn_based_strategy,turn_based_tactics,tutorial,twin_stick_shooter,typing,underground,underwater,unforgiving,utilities,vr,vr_only,vampire,video_production,villain_protagonist,violent,visual_novel,voice_control,voxel,walking_simulator,war,wargame,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
appid,,,,,,,,,,,,,

## Transforming Rating Data

To better utilize the rating data and incorporate the number of owners as a weighting method for the rating, the `rating_df` DataFrame is transformed as follows:

### Data Transformation Steps
1. Compute a new column `rating_score` by dividing the positive ratings by the total number of ratings.
2. Convert the 'owners' column into a normalized scale based on the number of owners.
3. Compute the `weighted_rating` by multiplying the `rating_score` and the normalized `owners` values.

The advantages of this approach include:
- The rating score provides a more meaningful metric to compare games by accounting for both positive and negative ratings.
- The weighted rating takes into consideration the number of owners, giving more weight to popular games and mitigating the impact of games with a smaller user base that may have skewed ratings.



In [29]:
rating_df['rating_score'] = rating_df['positive_ratings']/(rating_df['positive_ratings'] + 
                                                           rating_df['negative_ratings'])
rating_df.drop(columns=['positive_ratings', 'negative_ratings'], inplace=True)
rating_df.sample(5)

,owners,rating_score
551750,0-20000,0.785714
656740,0-20000,0.965517
957390,0-20000,0.750000
304650,500000-1000000,0.828567
776000,0-20000,0.636364


In [30]:
display(rating_df['owners'].value_counts())
rating_df['owners'] = rating_df['owners'].apply(lambda x: 0.6 if x=='0-20000' else 
                                                          0.8 if x=='20000-50000' else 
                                                          0.9 if x=='50000-100000' else 
                                                          1)
display(rating_df['owners'].value_counts())

rating_df['weighted_rating'] = rating_df['rating_score']*rating_df['owners']
rating_df.drop(columns=['owners','rating_score'], inplace=True)
rating_df.sample(5)

0-20000                18129
20000-50000             3012
50000-100000            1671
100000-200000           1368
200000-500000           1268
500000-1000000           512
1000000-2000000          284
2000000-5000000          191
5000000-10000000          45
10000000-20000000         21
20000000-50000000          3
50000000-100000000         2
100000000-200000000        1
Name: owners, dtype: int64

0.6    18129
1.0     3695
0.8     3012
0.9     1671
Name: owners, dtype: int64

,weighted_rating
731520,0.600000
342660,0.400000
231140,0.569004
523210,0.698113
778150,0.507692


## Transforming Release Date Data

To better utilize the release date data, the `date_df` DataFrame is transformed as follows:

### Data Transformation Steps
1. Convert the 'release_date' column to a datetime format.
2. Compute 'days_since_release' by calculating the number of days between the release date and the current date.
3. Normalize the 'days_since_release' column using MinMaxScaler.



In [31]:
date_df['release_date'] = pd.to_datetime(date_df['release_date'])

reference_date = datetime.now()
date_df['days_since_release'] = (reference_date - date_df['release_date']).dt.days

scaler = MinMaxScaler()
date_df['days_norm'] = scaler.fit_transform(date_df[['days_since_release']])

date_df.drop(columns=['release_date', 'days_since_release'], inplace=True)

date_df.sample(5)

,days_norm
461350,0.073480
972740,0.021066
884410,0.035862
968470,0.021693
615910,0.086646


## Cleaning and Transforming Developer Data

The `dev_df` DataFrame contains many developer names with variations, additional characters, and sub-branches.

After exloring all developers, in order to better utilize this data, the following steps are taken:

1. Remove any irrelevant characters from the developer names.
2. Combine sub-branches or variations of major developers, such as CAPCOM, 2K, and UBISOFT, into a single developer name.
3. Convert the cleaned string of developer names into a list to utilize Jaccard similarity later on.


In [32]:
dev_df.sample(5)

,developer
849870,"MAMMOSSIX Co., Ltd."
720250,Reflect Studios
50910,Big Fish Games
709350,Mad Head Games
494990,AK84C


In [33]:
def devlist(string):
    
    for char in '•.\',()?!/&\"+:[]_-{}®©':
        string = string.replace(char, '')

    for s in ['BANDAI NAMCO Entertainment Inc','BANDAI NAMCO Studio Inc','BANDAI NAMCO Studios Inc','BANDAI NAMCO Studios Vancouver','BANDAI NAMCO Studios']:
        string = string.replace(s, 'BANDAI NAMCO')
    for s in ['Aspyr Mac Linux  Windows Update','Aspyr Linux','Aspyr Mac  Linux','Aspyr Mac']:
        string = string.replace(s, 'Aspyr')
    for s in ['2K Australia','2K Boston','2K China','2K Czech','2K Marin']:
        string = string.replace(s, '2K')
    for s in ['Ubisoft  Shanghaï','Ubisoft - San Francisco','Ubisoft Annecy','Ubisoft Belgrade','Ubisoft Blue Byte','Ubisoft Bucharest','Ubisoft Bulgaria','Ubisoft Entertainment','Ubisoft Kiev','Ubisoft Milan','Ubisoft Montpellier','Ubisoft Montreal','Ubisoft Montreal Studio','Ubisoft Montreal, Massive Entertainment, and Ubisoft Shanghai','Ubisoft Montreal, Red Storm, Shanghai, Toronto, Kiev','Ubisoft Montréal','Ubisoft Paris','Ubisoft Pune','Ubisoft Quebec','Ubisoft Quebec, in collaboration with Ubisoft Annecy, Bucharest, Kiev, Montreal, Montpellier, Shanghai, Singapore, Sofia, Toronto studios','Ubisoft Reflections','Ubisoft Romania','Ubisoft San Francisco','Ubisoft Shanghai','Ubisoft Singapore','Ubisoft Sofia','Ubisoft Toronto']:
        string = string.replace(s, 'Ubisoft')
        
    for s in ['Rockstar Leeds','Rockstar New England','Rockstar North','Rockstar North / Toronto','Rockstar Studios','Rockstar Toronto']:        
        string = string.replace(s, 'Rockstar Games')

    string = string.replace('Alternative Software Ltd','Alternative Software')        
    string = string.replace('Alternative Dreams Studios','Alternative Dreams')        
    string = string.replace('ARTDINK CORPORATION', 'ARTDINK')        
    string = string.replace('4 Fun Studio Inc','4 Fun Studio')              
    string = string.replace('FrameLineNetwork Kft','FrameLineNetwork',)
    string = string.replace('Flight Systems LLC', 'Flight Systems',)
    string = string.replace('Feral Interactive MacLinux', 'Feral interactive')
    string = string.replace('Feral interactive Mac', 'Feral interactive')
    string = string.replace('Feral Interactive Linux', 'Feral interactive')
    string = string.replace('FarSight Studios Inc', 'FarSight Studios')
    string = string.replace('FELISTELLA Co Ltd','FELISTELLA')
    string = string.replace('ERS Game Studios','ERS GStudios')
    string = string.replace('ERS Games Studio','ERS GStudios')
    string = string.replace('Evil Tortilla Games Incorporated','Evil Tortilla Games')
    string = string.replace('ECC GAMES SP Z OO',  'ECC GAMES SA')
    string = string.replace('Deceptive Games Ltd', 'Deceptive Games')
    string = string.replace('CyberConnect2 Co Ltd', 'CyberConnect2')
    string = string.replace('CAPCOM CO LTD', 'CAPCOM')
    string = string.replace('CAPCOM Co Ltd', 'CAPCOM')
    string = string.replace('VRROOM Ultimate VR Experiences  BV', 'VRROOM Ultimate VR Experiences')
    string = string.replace('Subaltern Games LLC','Subaltern Games')
    string = string.replace('Stumphead GamesLLC','Stumphead Games')
    string = string.replace('Stainless GamesLtd','Stainless Games')
    string = string.replace('Square Enix Montréal', 'Square Enix')
    string = string.replace('Spark Plug Games LLC','Spark Plug Games')
    string = string.replace('Spaces of Play UG','Spaces of Play')
    string = string.replace('Sanzaru Games Inc','Sanzaru Games')
    string = string.replace('Rocketeer Games Studio LLC','Rocketcat Games')
    string = string.replace('Random Thoughts Enterainment','Random Thoughts Entertainment')
    string = string.replace('Napoleon Games sro', 'Napoleon Games')
    string = string.replace('Monolith Productions, Inc','Monolith Productions')
    string = string.replace('Modern Dream Ltd','Modern Dream')
    string = string.replace('McMagic Productions sro','McMagic Productions')
    string = string.replace('Kverta Limited', 'Kverta')
    string = string.replace('Kool2Play Sp z oo', 'Kool2Play')
    string = string.replace('Independent Arts Software GmbH','Independent Arts Software')
    string = string.replace('Immersive VR Education Ltd','Immersive VR Education PLC')
    string = string.replace('FromSoftware Inc', 'FromSoftware')
    string = string.replace('Frima Studio Inc','Frima')
    string = string.replace('Frima Studio','Frima')

    string = string.lower()
    string = string.replace(' ', '')
                              
    return string.split(";")

In [34]:
dev_df['dev_list'] = dev_df['developer'].apply(devlist)
dev_df.drop(columns=['developer'], inplace=True)

dev_df.sample(5)

,dev_list
7220,[pendulostudios]
464500,[frontwing]
428240,[sharktreestudios]
827270,[salsawi]
6420,"[mithisgames, thqnordic]"


## Cleaning and Transforming Publisher Data

Similar to the developer data, the `pub_df` DataFrame also contains variations and additional characters in the publisher names. Therefore, to better utilize this data, the same steps are taken:

1. Remove any irrelevant characters from the publisher names.
2. Combine sub-branches or variations of major publishers, such as CAPCOM, and UBISOFT, into a single publisher name.
3. Convert the cleaned string of publisher names into a list to utilize Jaccard similarity later on.


In [35]:
def publist(string):
    
    for char in '•.\',\\()?!/&\"#+:[]_-{}=%®©@$':
        string = string.replace(char, '')
        
    for s in ['Aspyr Mac Linux','Aspyr Linux','Aspyr Mac']:
        string = string.replace(s, 'Aspyr')

    for s in ['Bandai Namco', 'Bandai Namco Entertainment', 'BANDAI NAMCO Entertainment America', 'BANDAI NAMCO Entertainment Europe', 'BANDAI NAMCO Entertainment', 'BANDAI NAMCO Entertainement',]:
        string = string.replace(s, 'BANDAI NAMCO')
       
    for s in ['Bethesda Softworks','Bethesda-Soft','Bethesda-Softworks']:
        string = string.replace(s, 'Bethesda')
 
    for s in ['Dovetail Games - Fishing','Dovetail Games - Flight','Dovetail Games - TSW','Dovetail Games - Trains',]:
        string = string.replace(s, 'Dovetail Games')
        
    for s in ['CAPCOM CO LTD','CAPCOM Co Ltd','Capcom Co Ltd','Capcom USA Inc']:
        string = string.replace(s, 'CAPCOM')
  
    for s in ['Gaijin Distribution KFT','Gaijin Entertainment','Gaijin Entertainment Corporation','Gaijin inCubator']:
        string = string.replace(s, 'Gaijin Games')
    
    for s in ['Konami Digital Entertainement GmbH', 'Konami Digital Entertainment', 'Konami Digital Entertainment GmbH','Konami Digital Entertainment Inc']:
        string = string.replace(s, 'Konami')
   
    string = string.replace('Big Ant Studios Steam', 'Big Ant Studios')
    string = string.replace( 'Big Fish Games Inc',  'Big Fish Games')
    string = string.replace('Bitbox SL', 'Bitbox Ltd')
    string = string.replace('Blazing Griffin Ltd','Blazing Griffin')
    string = string.replace('Blob Games Studio','Blob Games')
    string = string.replace('CM Softworks Inc','CM Softworks')
    string = string.replace('Cartoon Network Games', 'Cartoon Network')
    string = string.replace('CasGames', 'CasGame')
    string = string.replace('Chorus Worldwide Games Limited','Chorus Worldwide')
    string = string.replace('Circle 5 Studios','Circle 5')
    string = string.replace('Crazy Rocks Studios','Crazy Rocks')
    string = string.replace('Crazysoft Limited','Crazysoft Ltd')
    string = string.replace('Crian Soft SA','Crian Soft')
    string = string.replace('Empyrean Interactive','Empyrean')
    string = string.replace('EuroVideo Medien GmbH','EuroVideo Medien')
    string = string.replace('Fair Games Studio','Fair Games')
    string = string.replace('Fantasy Flight Publishing Inc', 'Fantasy Flight Publishing')
    string = string.replace('FarSight Studios Inc',  'FarSight Studios')
    string = string.replace('Fatbot Games s r o', 'Fatbot Games')
    string = string.replace('Fatmoth Interactive','Fatmoth')
    string = string.replace('Feral Interactive MacLinux', 'Feral Interactive')
    string = string.replace('Feral Interactive Linux', 'Feral Interactive')
    string = string.replace('Feral Interactive Mac', 'Feral Interactive')
    string = string.replace('Five Mind Creations UG haftungsbeschraenkt','Five Mind Creations')
    string = string.replace('Fixpoint Productions Ltd', 'Fixpoint Productions')
    string = string.replace('Flight Systems LLC','Flight Systems')
    string = string.replace('Forthright Entertainment LLC','Forthright Entertainment')
    string = string.replace('Fossil Games','Fossil')
    string = string.replace('Frima Originals','Frima')
    string = string.replace('Frima Studio','Frima')
    string = string.replace('FromSoftware Japan','FromSoftware')
    string = string.replace('FromSoftware Inc','FromSoftware')
    string = string.replace('Game Troopers SL','Game Troopers')
    string = string.replace('Gameforge 4D GmbHu202c', 'Gameforge 4D GmbH')
    string = string.replace('GungHo Online Entertainment America Inc','GungHo Online Entertainment America')
    string = string.replace('Hazardous Software Inc','Hazardous Software')
    string = string.replace('Idea Factory International Inc','Idea Factory')
    string = string.replace('Idea Factory International','Idea Factory')
    string = string.replace('Immanitas Entertainment GmbH','Immanitas Entertainment')
    string = string.replace('Immanitas Entertainment GmbH','Immanitas Entertainment PLC')
    string = string.replace('Kagura Games Chinese Localization','Kagura Games')
    string = string.replace('Kerberos Productions Inc','Kerberos Productions')
    string = string.replace('Kool2Play Sp z oo','Kool2Play')
    string = string.replace('LB Studios','LB')
    string = string.replace('Lemondo Entertainment', 'Lemondo Games')
    string = string.replace('Lofty','Loft')
    string = string.replace('MAGES Inc','MAGES')
    string = string.replace('MGP Studios', 'MGP')
    string = string.replace('MK game production', 'MK Games')
    string = string.replace('MK-ULTRA Games', 'MK Games')
    string = string.replace('MLBcom','MLB')
    string = string.replace('Mayflower Entertainment KR','Mayflower Entertainment')
    string = string.replace('McMagic Productions sro','McMagic Productions')
    string = string.replace('NS STUDIO','NS')
    string = string.replace('Nexon America Inc','Nexon',)
    string = string.replace('Nexon America','Nexon',)
    string = string.replace('Nexon Korea Corporation','Nexon')
    string = string.replace('Oddworld Inhabitants Inc','Oddworld Inhabitants')
    string = string.replace('Outright Games Ltd', 'Outright Games')
    string = string.replace('Perfect Square Studios LLC','Perfect Square Studios')
    string = string.replace('Praxia Entertainment Inc','Praxia Entertainment')
    string = string.replace('SelfPublished','Selfp')
    string = string.replace('Ubisoft Entertainment','Ubisoft')
    string = string.replace( 'Viva Media Inc', 'Viva Media')
    string = string.replace('Warner Bros Interactive Entertainment','Warner Bros')
    string = string.replace('Warner Bros Interactive','Warner Bros')
    string = string.replace('crowgames UG haftungsbeschränkt','crowgames')
    string = string.replace('方块游戏 Asia', '方块游戏')
    string = string.replace('方块游戏CubeGame', '方块游戏')

    string = string.lower()
    string = string.replace(' ', '')

    if string == '':
        string = 'unknown'
        
    return string.split(";")

In [36]:
pub_df['pub_list'] = pub_df['publisher'].apply(publist)
pub_df.drop(columns=['publisher'], inplace=True)
pub_df.sample(5)

,pub_list
985930,[watertemplestudio]
502500,[bandainamco]
881250,[fyrg]
544790,[aelentertainment]
997280,[flashynurav]


## Processing the description dataframe

The des_df DataFrame contains the 'about_the_game' descriptions for the games. To convert these descriptions into numerical vectors that can be used for similarity calculations, I used the doc.vector to call tex embedding vectorizer function, from  the SpaCy library, obtaining a 300-dimensional  vector representation of text data. This method is useful because it takes into account the context and the meaning of the words.

Using SpaCy's doc.vector is useful because it provides a compact numerical representation of the text, which can be utilized for similarity calculations in the recommendation system.

SpaCy is a popular open-source library for natural language processing in Python. It is designed to be fast, efficient, and easy to use, offering various capabilities such as tokenization, part-of-speech tagging, dependency parsing, and more.

The advantage of using SpaCy for processing game descriptions is that it simplifies the process of converting text data into numerical vectors while maintaining the semantic relationships between the words. This allows the recommendation system to effectively capture the similarities between games based on their descriptions.

In [37]:
def get_vector(text):
    doc = nlp(text)
    return doc.vector

start_time = time()

des_vectors = des_df['about_the_game'].apply(get_vector)

print(f"Elapsed time: {time()-start_time} seconds")

des_vectors_df = pd.DataFrame(des_vectors.to_list())
des_vectors_df = des_vectors_df.rename(columns={(i): f'description V_{i+1}' for i in range(0, len(des_vectors_df.columns)+1)})
des_vectors_df.index = des_df.index
des_vectors_df.sample(5)

Elapsed time: 839.2062339782715 seconds


,description V_1,description V_2,description V_3,description V_4,description V_5,description V_6,description V_7,description V_8,description V_9,description V_10,description V_11,description V_12,description V_13,description V_14,description V_15,description V_16,description V_17,description V_18,description V_19,description V_20,description V_21,description V_22,description V_23,description V_24,description V_25,description V_26,description V_27,description V_28,description V_29,description V_30,description V_31,description V_32,description V_33,description V_34,description V_35,description V_36,description V_37,description V_38,description V_39,description V_40,description V_41,description V_42,description V_43,description V_44,description V_45,description V_46,description V_47,description V_48,description V_49,description V_50,description V_51,description V_52,description V_53,description V_54,description V_55,description V_56,description V_57,description V_58,description V_59,description V_60,description V_61,description V_62,description V_63,description V_64,description V_65,description V_66,description V_67,description V_68,description V_69,description V_70,description V_71,description V_72,description V_73,description V_74,description V_75,description V_76,description V_77,description V_78,description V_79,description V_80,description V_81,description V_82,description V_83,description V_84,description V_85,description V_86,description V_87,description V_88,description V_89,description V_90,description V_91,description V_92,description V_93,description V_94,description V_95,description V_96,description V_97,description V_98,description V_99,description V_100,description V_101,description V_102,description V_103,description V_104,description V_105,description V_106,description V_107,description V_108,description V_109,description V_110,description V_111,description V_112,description V_113,description V_114,description V_115,description V_116,description V_117,description V_118,description V_119,description V_120,description V_121,description V_122,description V_123,description V_124,description V_125,description V_126,description V_127,description V_128,description V_129,description V_130,description V_131,description V_132,description V_133,description V_134,description V_135,description V_136,description V_137,description V_138,description V_139,description V_140,description V_141,description V_142,description V_143,description V_144,description V_145,description V_146,description V_147,description V_148,description V_149,description V_150,description V_151,description V_152,description V_153,description V_154,description V_155,description V_156,description V_157,description V_158,description V_159,description V_160,description V_161,description V_162,description V_163,description V_164,description V_165,description V_166,description V_167,description V_168,description V_169,description V_170,description V_171,description V_172,description V_173,description V_174,description V_175,description V_176,description V_177,description V_178,description V_179,description V_180,description V_181,description V_182,description V_183,description V_184,description V_185,description V_186,description V_187,description V_188,description V_189,description V_190,description V_191,description V_192,description V_193,description V_194,description V_195,description V_196,description V_197,description V_198,description V_199,description V_200,description V_201,description V_202,description V_203,description V_204,description V_205,description V_206,description V_207,description V_208,description V_209,description V_210,description V_211,description V_212,description V_213,description V_214,description V_215,description V_216,description V_217,description V_218,description V_219,description V_220,description V_221,description V_222,description V_223,description V_224,description V_225,description V_226,description V_227,description V_228,des

In [38]:
des_vectors_df[des_vectors_df.isna().any(axis=1)]

,description V_1,description V_2,description V_3,description V_4,description V_5,description V_6,description V_7,description V_8,description V_9,description V_10,description V_11,description V_12,description V_13,description V_14,description V_15,description V_16,description V_17,description V_18,description V_19,description V_20,description V_21,description V_22,description V_23,description V_24,description V_25,description V_26,description V_27,description V_28,description V_29,description V_30,description V_31,description V_32,description V_33,description V_34,description V_35,description V_36,description V_37,description V_38,description V_39,description V_40,description V_41,description V_42,description V_43,description V_44,description V_45,description V_46,description V_47,description V_48,description V_49,description V_50,description V_51,description V_52,description V_53,description V_54,description V_55,description V_56,description V_57,description V_58,description V_59,description V_60,description V_61,description V_62,description V_63,description V_64,description V_65,description V_66,description V_67,description V_68,description V_69,description V_70,description V_71,description V_72,description V_73,description V_74,description V_75,description V_76,description V_77,description V_78,description V_79,description V_80,description V_81,description V_82,description V_83,description V_84,description V_85,description V_86,description V_87,description V_88,description V_89,description V_90,description V_91,description V_92,description V_93,description V_94,description V_95,description V_96,description V_97,description V_98,description V_99,description V_100,description V_101,description V_102,description V_103,description V_104,description V_105,description V_106,description V_107,description V_108,description V_109,description V_110,description V_111,description V_112,description V_113,description V_114,description V_115,description V_116,description V_117,description V_118,description V_119,description V_120,description V_121,description V_122,description V_123,description V_124,description V_125,description V_126,description V_127,description V_128,description V_129,description V_130,description V_131,description V_132,description V_133,description V_134,description V_135,description V_136,description V_137,description V_138,description V_139,description V_140,description V_141,description V_142,description V_143,description V_144,description V_145,description V_146,description V_147,description V_148,description V_149,description V_150,description V_151,description V_152,description V_153,description V_154,description V_155,description V_156,description V_157,description V_158,description V_159,description V_160,description V_161,description V_162,description V_163,description V_164,description V_165,description V_166,description V_167,description V_168,description V_169,description V_170,description V_171,description V_172,description V_173,description V_174,description V_175,description V_176,description V_177,description V_178,description V_179,description V_180,description V_181,description V_182,description V_183,description V_184,description V_185,description V_186,description V_187,description V_188,description V_189,description V_190,description V_191,description V_192,description V_193,description V_194,description V_195,description V_196,description V_197,description V_198,description V_199,description V_200,description V_201,description V_202,description V_203,description V_204,description V_205,description V_206,description V_207,description V_208,description V_209,description V_210,description V_211,description V_212,description V_213,description V_214,description V_215,description V_216,description V_217,description V_218,description V_219,description V_220,description V_221,description V_222,description V_223,description V_224,description V_225,description V_226,description V_227,description V_228,des

## Storage of Precomputed Tables

The proposed recommendation system pipeline relies on the availability of precomputed, normalized tables. <br>
This approach is particularly beneficial due to the time-consuming nature of selecting English-only content and  Spacy doc to vectors. 

In order to efficiently utilize these tables, they can be computed beforehand and stored alongside the standard Steam tables provided. This will facilitate seamless incorporation of updated scores whenever any changes occur within the dataset. 

For the purpose of storing and retrieving these tables, the Pickle library will be utilized within the pipeline.

In [39]:
stored_tables = {'scores_df': scores_df, 
                 'dev_df': dev_df, 
                 'pub_df': pub_df,
                 'tag_df': tag_df,
                 'gen_df': gen_df, 
                 'cat_df': cat_df,
                 'rating_df': rating_df,
                 'date_df': date_df,
                 'des_vectors_df': des_vectors_df,
                 'name_df': name_df,
                 'title': df['name'],
                 'rel_date': df['release_date']}


with open('steam_eng_tables', 'wb') as f:
    pickle.dump(stored_tables, f)

# Pipeline

In [40]:
import numpy as np
import pandas as pd
import pickle

with open('steam_eng_tables', 'rb') as f:
    stored_tables = pickle.load(f)

## Calculating Similarity Scores

Functions to calculate similarity scores were created for each DataFrame using different methods based on the nature of the information stored:

- **Developers and Publishers DataFrames**: We calculated a score based on the number of common developers or publishers between the target game and the compared game, taking into account the total number of developers or publishers for each. A score of 1 is given if all developers or publishers are the same, 0.8 if at least half of the developers or publishers are the same, 0.5 if there is at least one common developer or publisher, and 0 otherwise. 


- **Tags, Genres, and Categories DataFrames**: We used the Jaccard similarity index to calculate a score based on the similarity of the tags, genres, or categories between the target game and the compared game. Having a multilabel list of tags and genres fits well with Jaccard similarity because it is a measure designed to compare sets, making it a suitable choice for situations where data is in the form of unordered lists.


- **Description and Tags DataFrames**: The cosine similarity is used to calculate a score based on the similarity of the game descriptions between the target game and the compared game. Having a large number of standardized continuous values between 0 and 1 or -1 and 1 fits well with cosine similarity because it effectively captures the angle between two vectors, providing a meaningful measure of similarity of the vectors.


- **Release Date DataFrame**: We used a simple score based on the proximity of the release date of the target game and the compared game. A smoothing factor has been added, applying the power of a float number between 0 and 1 to the original score (between 0 and 1) to smooth the values. This approach ensures that small differences in release dates do not lead to a significant impact on the overall similarity score.


- **Rating DataFrame**: The rating DataFrame stayed the same because it had been normalized previously, and the weighted rating already provides a meaningful metric for comparison.


In [41]:
def calculate_dev_score(game_devs, other_game_devs):
    intersection = set(game_devs) & set(other_game_devs)
    num_common_devs = len(intersection)
    num_devs = len(game_devs)
    num_other_dev = len(other_game_devs)
    
    if num_common_devs == num_devs and num_common_devs == num_other_dev:
        return 1
    elif num_common_devs > 0 and num_common_devs >= (num_devs / 2):
        return 0.8
    elif num_common_devs > 0:
        return 0.5
    else:
        return 0

def get_dev_scores(df,game_index):
    target = df.loc[game_index][0]
    dev_scores = dev_df.iloc[:,0].apply(lambda x: calculate_dev_score(target, x))
    df.columns = ['dev_score']
    return dev_scores.to_frame()

get_dev_scores(dev_df, 10).T



,10,20,30,40,50,60,70,80,130,220,240,280,300,320,340,360,380,400,420,440,500,550,570,620,630,730,1002,1200,1250,1300,1500,1510,1520,1530,1600,1610,1630,1640,1670,1690,1700,1840,1900,1930,2100,2200,2210,2270,2280,2290,2300,2310,2320,2330,2340,2350,2360,2370,2390,2400,2450,2500,2520,2590,2600,2610,2620,2630,2640,2710,2720,2780,2800,2810,2820,2840,2850,2870,2900,2910,2920,2990,3010,3020,3050,3130,3170,3230,3260,3270,3330,3410,3480,3483,3490,3500,3510,3520,3530,3540,3560,3570,3580,3590,3600,3610,3620,3700,3710,3720,3730,3800,3810,3820,3830,3900,3910,3920,3960,3980,3990,4000,4100,4230,4290,4300,4420,4460,4470,4500,4520,4530,4560,4570,4580,4700,4720,4760,4770,4780,4800,4850,4870,4880,4890,4900,4920,6000,6010,6020,6030,6040,6060,6080,6090,6120,6200,6210,6220,6250,6270,6300,6310,6370,6400,6420,6510,6550,6800,6810,6830,6840,6850,6860,6870,6880,6900,6910,6920,6980,7000,7010,7020,7110,7200,7210,7220,7260,7340,7510,7520,7530,7600,7610,7620,7650,7660,7730,7760,7770,...,1043480,1043500,1043510,1043560,1043580,1043610,1043680,1043730,1043740,1043890,1044170,1044200,1044240,1044340,1044350,1044450,1044530,1044630,1044640,1044770,1044830,1044840,1044920,1044950,1045020,1045080,1045220,1045300,1045530,1045740,1045850,1045930,1046030,1046070,1046110,1046230,1046240,1046330,1046370,1046430,1046490,1046530,1046560,1046670,1046750,1046820,1047120,1047140,1047160,1047190,1047240,1047670,1047720,1047780,1047910,1047960,1048000,1048040,1048080,1048100,1048170,1048260,1048410,1048470,1048570,1048640,1048830,1048850,1048920,1048960,1049070,1049080,1049090,1049140,1049230,1049270,1049370,1049420,1049660,1049680,1049730,1049800,1049840,1049910,1049930,1049950,1050010,1050150,1050190,1050210,1050230,1050240,1050430,1050470,1050690,1050730,1050760,1050870,1051130,1051160,1051170,1051250,1051280,1051310,1051500,1051530,1051810,1051830,1051840,1051890,1052010,1052070,1052220,1052480,1052760,1052850,1052870,1052900,1053040,1053060,1053090,1053160,1053190,1053250,1053300,1053650,1053660,1053680,1053730,1053740,1053780,1053960,1054240,1054250,1054560,1054650,1054750,1054790,1054900,1054930,1054980,1055000,1055090,1055140,1055430,1055620,1055690,1055770,1055890,1055970,1055990,1056260,1056470,1056660,1057180,1057390,1057420,1057430,1057460,1057500,1057660,1057690,1057710,1058150,1058350,1058430,1058590,1058660,1058910,1058930,1058940,1059090,1059190,1059280,1059500,1059710,1060030,1060110,1060170,1060300,1060440,1060770,1060870,1061230,1061470,1062120,1062670,1062880,1063060,1063230,1063560,1064060,1064580,1064890,1065160,1065230,1065570,1065650,1066700,1069460
dev_list,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [42]:
def calculate_pub_score(game_pubs, other_game_pubs):
    intersection = set(game_pubs) & set(other_game_pubs)
    num_common_pubs = len(intersection)
    num_pubs = len(game_pubs)
    num_other_pub = len(other_game_pubs)
    
    if num_common_pubs == num_pubs and num_common_pubs == num_other_pub:
        return 1
    elif num_common_pubs > 0 and num_common_pubs >= (num_pubs / 2):
        return 0.8
    elif num_common_pubs > 0:
        return 0.5
    else:
        return 0

def get_pub_scores(df,game_index):
    target = df.loc[game_index][0]
    pub_scores = pub_df.iloc[:,0].apply(lambda x: calculate_pub_score(target, x))
    df.columns = ['pub_score']
    return pub_scores.to_frame()

get_pub_scores(pub_df, 10).T

,10,20,30,40,50,60,70,80,130,220,240,280,300,320,340,360,380,400,420,440,500,550,570,620,630,730,1002,1200,1250,1300,1500,1510,1520,1530,1600,1610,1630,1640,1670,1690,1700,1840,1900,1930,2100,2200,2210,2270,2280,2290,2300,2310,2320,2330,2340,2350,2360,2370,2390,2400,2450,2500,2520,2590,2600,2610,2620,2630,2640,2710,2720,2780,2800,2810,2820,2840,2850,2870,2900,2910,2920,2990,3010,3020,3050,3130,3170,3230,3260,3270,3330,3410,3480,3483,3490,3500,3510,3520,3530,3540,3560,3570,3580,3590,3600,3610,3620,3700,3710,3720,3730,3800,3810,3820,3830,3900,3910,3920,3960,3980,3990,4000,4100,4230,4290,4300,4420,4460,4470,4500,4520,4530,4560,4570,4580,4700,4720,4760,4770,4780,4800,4850,4870,4880,4890,4900,4920,6000,6010,6020,6030,6040,6060,6080,6090,6120,6200,6210,6220,6250,6270,6300,6310,6370,6400,6420,6510,6550,6800,6810,6830,6840,6850,6860,6870,6880,6900,6910,6920,6980,7000,7010,7020,7110,7200,7210,7220,7260,7340,7510,7520,7530,7600,7610,7620,7650,7660,7730,7760,7770,...,1043480,1043500,1043510,1043560,1043580,1043610,1043680,1043730,1043740,1043890,1044170,1044200,1044240,1044340,1044350,1044450,1044530,1044630,1044640,1044770,1044830,1044840,1044920,1044950,1045020,1045080,1045220,1045300,1045530,1045740,1045850,1045930,1046030,1046070,1046110,1046230,1046240,1046330,1046370,1046430,1046490,1046530,1046560,1046670,1046750,1046820,1047120,1047140,1047160,1047190,1047240,1047670,1047720,1047780,1047910,1047960,1048000,1048040,1048080,1048100,1048170,1048260,1048410,1048470,1048570,1048640,1048830,1048850,1048920,1048960,1049070,1049080,1049090,1049140,1049230,1049270,1049370,1049420,1049660,1049680,1049730,1049800,1049840,1049910,1049930,1049950,1050010,1050150,1050190,1050210,1050230,1050240,1050430,1050470,1050690,1050730,1050760,1050870,1051130,1051160,1051170,1051250,1051280,1051310,1051500,1051530,1051810,1051830,1051840,1051890,1052010,1052070,1052220,1052480,1052760,1052850,1052870,1052900,1053040,1053060,1053090,1053160,1053190,1053250,1053300,1053650,1053660,1053680,1053730,1053740,1053780,1053960,1054240,1054250,1054560,1054650,1054750,1054790,1054900,1054930,1054980,1055000,1055090,1055140,1055430,1055620,1055690,1055770,1055890,1055970,1055990,1056260,1056470,1056660,1057180,1057390,1057420,1057430,1057460,1057500,1057660,1057690,1057710,1058150,1058350,1058430,1058590,1058660,1058910,1058930,1058940,1059090,1059190,1059280,1059500,1059710,1060030,1060110,1060170,1060300,1060440,1060770,1060870,1061230,1061470,1062120,1062670,1062880,1063060,1063230,1063560,1064060,1064580,1064890,1065160,1065230,1065570,1065650,1066700,1069460
pub_list,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
# normalize the vector lenghts to 1, needed to calculate cosine similarity
def normalize_df(df):
    norm = np.linalg.norm(df, axis=1)[:, np.newaxis]
    return df / norm

# calculate cosine similarity ongly for the target row, instead of the whole matrix
def single_row_cosine_similarity(df, target_row, title):
    normalized_df = normalize_df(df)
    target_row_normalized = normalized_df.loc[target_row]
    cosine_similarities = normalized_df @ target_row_normalized
    
    return pd.DataFrame(cosine_similarities, columns = [title])


display(single_row_cosine_similarity(tag_df, 10, 'tag_score').T)
display(single_row_cosine_similarity(des_vectors_df, 10, 'des_score').T)

appid,10,20,30,40,50,60,70,80,130,220,240,280,300,320,340,360,380,400,420,440,500,550,570,620,630,730,1002,1200,1250,1300,1309,1313,1500,1510,1520,1530,1600,1610,1630,1640,1670,1690,1700,1840,1900,1930,2100,2200,2210,2270,2280,2290,2300,2310,2320,2330,2340,2350,2360,2370,2390,2400,2420,2450,2500,2520,2540,2570,2590,2600,2610,2620,2630,2640,2700,2710,2720,2780,2800,2810,2820,2840,2850,2870,2900,2910,2920,2990,3010,3020,3050,3130,3170,3230,3260,3270,3300,3310,3320,3330,3340,3350,3360,3380,3390,3400,3410,3420,3430,3440,3450,3460,3480,3483,3490,3500,3510,3520,3530,3540,3560,3570,3580,3590,3600,3610,3620,3700,3710,3720,3730,3800,3810,3820,3830,3900,3910,3920,3960,3980,3990,4000,4100,4230,4290,4300,4420,4460,4470,4500,4520,4530,4560,4570,4580,4700,4720,4760,4770,4780,4800,4850,4870,4880,4890,4900,4920,6000,6010,6020,6030,6040,6060,6080,6090,6120,6200,6210,6220,6250,6270,6300,6310,6370,6400,6420,6510,6550,6600,6800,6810,6830,6840,6850,6860,6870,6880,6900,6910,6920,...,1044340,1044350,1044450,1044530,1044630,1044640,1044770,1044830,1044840,1044920,1044950,1045010,1045020,1045080,1045130,1045140,1045220,1045300,1045530,1045650,1045740,1045850,1045930,1046030,1046070,1046110,1046230,1046240,1046330,1046370,1046430,1046490,1046530,1046560,1046670,1046750,1046770,1046820,1047120,1047140,1047160,1047190,1047240,1047670,1047680,1047720,1047780,1047910,1047960,1048000,1048040,1048080,1048100,1048170,1048260,1048320,1048410,1048470,1048570,1048640,1048830,1048850,1048920,1048960,1049040,1049070,1049080,1049090,1049140,1049230,1049270,1049370,1049420,1049660,1049680,1049730,1049800,1049840,1049910,1049930,1049950,1050010,1050150,1050190,1050210,1050230,1050240,1050430,1050470,1050690,1050730,1050760,1050870,1051130,1051160,1051170,1051250,1051280,1051310,1051500,1051530,1051810,1051830,1051840,1051890,1052010,1052070,1052220,1052480,1052760,1052850,1052870,1052900,1053040,1053060,1053090,1053160,1053190,1053250,1053300,1053650,1053660,1053680,1053730,1053740,1053780,1053960,1054240,1054250,1054560,1054650,1054750,1054790,1054900,1054930,1054980,1055000,1055090,1055140,1055430,1055620,1055690,1055770,1055890,1055970,1055990,1056010,1056260,1056470,1056500,1056660,1056710,1057180,1057390,1057420,1057430,1057460,1057500,1057660,1057690,1057710,1058000,1058150,1058350,1058430,1058590,1058660,1058910,1058930,1058940,1059090,1059190,1059280,1059500,1059710,1060030,1060110,1060170,1060300,1060440,1060770,1060870,1061230,1061470,1062120,1062240,1062670,1062880,1063060,1063230,1063560,1064060,1064580,1064890,1065160,1065230,1065570,1065650,1066700,1069460
tag_score,1.0,0.891156,0.776308,0.904292,0.690284,0.792197,0.726906,0.931129,0.665625,0.640264,0.916603,0.695079,0.767209,0.814604,0.639661,0.862575,0.594849,0.254944,0.636142,0.602868,0.598991,0.557709,0.282683,0.240848,0.396257,0.839739,0.060527,0.689954,0.559951,0.711145,0.702534,0.702534,0.092829,0.070651,0.168675,0.145357,0.098314,0.051564,0.11766,0.16696,0.055965,0.077115,0.237902,0.09098,0.104958,0.179465,0.405826,0.818104,0.754705,0.722644,0.674439,0.754617,0.708495,0.778428,0.768244,0.801619,0.796759,0.860224,0.685159,0.78465,0.72819,0.508267,0.394593,0.768205,0.462547,0.0,0.199774,0.0,0.722086,0.179777,0.357376,0.735132,0.727403,0.732388,0.074445,0.46261,0.383483,0.696776,0.116421,0.084983,0.098045,0.076177,0.095356,0.143625,0.055868,0.065379,0.057562,0.297489,0.07901,0.587035,0.0,0.055837,0.047751,0.062019,0.023881,0.79677,0.0,0.0,0.171936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.138078,0.0,0.0,0.0,0.0,0.0,0.027558,0.080214,0.0,0.0,0.0,0.0,0.0,0.0,0.036365,0.0,0.0,0.062307,0.0,0.0,0.292503,0.512285,0.724974,0.058072,0.710309,0.446946,0.361765,0.359216,0.212164,0.152255,0.176574,0.125584,0.067786,0.051764,0.06958,0.375346,0.0,0.0,0.0,0.387927,0.071383,0.046246,0.076032,0.403727,0.476985,0.467016,0.336281,0.22221,0.221519,0.21342,0.406992,0.226232,0.099047,0.137993,0.056085,0.270808,0.06541,0.065355,0.062641,0.0,0.683877,0.654513,0.12687,0.551265,0.639317,0.101432,0.616421,0.251049,0.58454,0.319625,0.14

,10,20,30,40,50,60,70,80,130,220,240,280,300,320,340,360,380,400,420,440,500,550,570,620,630,730,1002,1200,1250,1300,1500,1510,1520,1530,1600,1610,1630,1640,1670,1690,1700,1840,1900,1930,2100,2200,2210,2270,2280,2290,2300,2310,2320,2330,2340,2350,2360,2370,2390,2400,2450,2500,2520,2590,2600,2610,2620,2630,2640,2710,2720,2780,2800,2810,2820,2840,2850,2870,2900,2910,2920,2990,3010,3020,3050,3130,3170,3230,3260,3270,3330,3410,3480,3483,3490,3500,3510,3520,3530,3540,3560,3570,3580,3590,3600,3610,3620,3700,3710,3720,3730,3800,3810,3820,3830,3900,3910,3920,3960,3980,3990,4000,4100,4230,4290,4300,4420,4460,4470,4500,4520,4530,4560,4570,4580,4700,4720,4760,4770,4780,4800,4850,4870,4880,4890,4900,4920,6000,6010,6020,6030,6040,6060,6080,6090,6120,6200,6210,6220,6250,6270,6300,6310,6370,6400,6420,6510,6550,6800,6810,6830,6840,6850,6860,6870,6880,6900,6910,6920,6980,7000,7010,7020,7110,7200,7210,7220,7260,7340,7510,7520,7530,7600,7610,7620,7650,7660,7730,7760,7770,...,1043480,1043500,1043510,1043560,1043580,1043610,1043680,1043730,1043740,1043890,1044170,1044200,1044240,1044340,1044350,1044450,1044530,1044630,1044640,1044770,1044830,1044840,1044920,1044950,1045020,1045080,1045220,1045300,1045530,1045740,1045850,1045930,1046030,1046070,1046110,1046230,1046240,1046330,1046370,1046430,1046490,1046530,1046560,1046670,1046750,1046820,1047120,1047140,1047160,1047190,1047240,1047670,1047720,1047780,1047910,1047960,1048000,1048040,1048080,1048100,1048170,1048260,1048410,1048470,1048570,1048640,1048830,1048850,1048920,1048960,1049070,1049080,1049090,1049140,1049230,1049270,1049370,1049420,1049660,1049680,1049730,1049800,1049840,1049910,1049930,1049950,1050010,1050150,1050190,1050210,1050230,1050240,1050430,1050470,1050690,1050730,1050760,1050870,1051130,1051160,1051170,1051250,1051280,1051310,1051500,1051530,1051810,1051830,1051840,1051890,1052010,1052070,1052220,1052480,1052760,1052850,1052870,1052900,1053040,1053060,1053090,1053160,1053190,1053250,1053300,1053650,1053660,1053680,1053730,1053740,1053780,1053960,1054240,1054250,1054560,1054650,1054750,1054790,1054900,1054930,1054980,1055000,1055090,1055140,1055430,1055620,1055690,1055770,1055890,1055970,1055990,1056260,1056470,1056660,1057180,1057390,1057420,1057430,1057460,1057500,1057660,1057690,1057710,1058150,1058350,1058430,1058590,1058660,1058910,1058930,1058940,1059090,1059190,1059280,1059500,1059710,1060030,1060110,1060170,1060300,1060440,1060770,1060870,1061230,1061470,1062120,1062670,1062880,1063060,1063230,1063560,1064060,1064580,1064890,1065160,1065230,1065570,1065650,1066700,1069460
des_score,1.0,0.961512,0.945795,0.934382,0.943008,0.960876,0.957626,0.921382,0.939103,0.95993,0.921551,0.927461,0.94593,0.90659,0.898946,0.914862,0.911405,0.95099,0.931475,0.970041,0.962765,0.967606,0.970802,0.963702,0.954715,0.934344,0.943194,0.945811,0.961023,0.978808,0.925389,0.951446,0.974835,0.969522,0.906787,0.950787,0.940282,0.957382,0.963802,0.962142,0.960466,0.946349,0.942822,0.971871,0.967026,0.960562,0.95777,0.955703,0.950295,0.952328,0.958971,0.909103,0.969787,0.924597,0.95802,0.960333,0.948411,0.928311,0.962551,0.973973,0.969066,0.964309,0.942311,0.96016,0.948947,0.962135,0.966577,0.976587,0.951485,0.958576,0.97249,0.965662,0.949014,0.94475,0.957397,0.965393,0.956783,0.975227,0.930475,0.965291,0.955009,0.947873,0.961526,0.952512,0.950183,0.944986,0.962261,0.969044,0.975305,0.953375,0.962711,0.958304,0.961449,0.955212,0.956311,0.942178,0.958476,0.926531,0.942921,0.961367,0.968038,0.950098,0.955279,0.952408,0.960756,0.948867,0.952486,0.961579,0.956144,0.967617,0.943873,0.957578,0.950428,0.939503,0.947423,0.962539,0.930562,0.963361,0.959207,0.966648,0.959322,0.975176,0.959713,0.950691,0.910562,0.94832,0.963569,0.963666,0.958741,0.960476,0.970293,0.965641,0.974327,0.967185,0.966052,0.95497,0.965087,0.949277,0.928205,0.944663,0.958683,0.950566,0.925063,0.954232,0.95542,0.946145,0.964523,0.974514,0.957152,0.957979,0.946351,0.958036,0.948665,0.962144,0.947758,0.952939,0.960638,0.951802,0.945193,0.974

In [44]:
# funtion that calculate the Jaccard similarity
def jaccard_similarity(target, row2):
    intersection = len(set(target)&set(row2))
    union = len(set(target).union(set(row2)))
    return intersection / union

# Calculate the Jaccard similarity for one row instead of the whole matrix
def single_row_jaccard_similarity(df, target_row, title):
    similarities = df.iloc[:,0].apply(lambda x: jaccard_similarity(
                                                df.loc[target_row][0], x))
    similarities = similarities.to_frame()
    similarities.columns = [title]
    return similarities

display(single_row_jaccard_similarity(gen_df, 10, 'gen_score').T)
display(single_row_jaccard_similarity(cat_df, 10, 'cat_score').T)


,10,20,30,40,50,60,70,80,130,220,240,280,300,320,340,360,380,400,420,440,500,550,570,620,630,730,1002,1200,1250,1300,1500,1510,1520,1530,1600,1610,1630,1640,1670,1690,1700,1840,1900,1930,2100,2200,2210,2270,2280,2290,2300,2310,2320,2330,2340,2350,2360,2370,2390,2400,2450,2500,2520,2590,2600,2610,2620,2630,2640,2710,2720,2780,2800,2810,2820,2840,2850,2870,2900,2910,2920,2990,3010,3020,3050,3130,3170,3230,3260,3270,3330,3410,3480,3483,3490,3500,3510,3520,3530,3540,3560,3570,3580,3590,3600,3610,3620,3700,3710,3720,3730,3800,3810,3820,3830,3900,3910,3920,3960,3980,3990,4000,4100,4230,4290,4300,4420,4460,4470,4500,4520,4530,4560,4570,4580,4700,4720,4760,4770,4780,4800,4850,4870,4880,4890,4900,4920,6000,6010,6020,6030,6040,6060,6080,6090,6120,6200,6210,6220,6250,6270,6300,6310,6370,6400,6420,6510,6550,6800,6810,6830,6840,6850,6860,6870,6880,6900,6910,6920,6980,7000,7010,7020,7110,7200,7210,7220,7260,7340,7510,7520,7530,7600,7610,7620,7650,7660,7730,7760,7770,...,1043480,1043500,1043510,1043560,1043580,1043610,1043680,1043730,1043740,1043890,1044170,1044200,1044240,1044340,1044350,1044450,1044530,1044630,1044640,1044770,1044830,1044840,1044920,1044950,1045020,1045080,1045220,1045300,1045530,1045740,1045850,1045930,1046030,1046070,1046110,1046230,1046240,1046330,1046370,1046430,1046490,1046530,1046560,1046670,1046750,1046820,1047120,1047140,1047160,1047190,1047240,1047670,1047720,1047780,1047910,1047960,1048000,1048040,1048080,1048100,1048170,1048260,1048410,1048470,1048570,1048640,1048830,1048850,1048920,1048960,1049070,1049080,1049090,1049140,1049230,1049270,1049370,1049420,1049660,1049680,1049730,1049800,1049840,1049910,1049930,1049950,1050010,1050150,1050190,1050210,1050230,1050240,1050430,1050470,1050690,1050730,1050760,1050870,1051130,1051160,1051170,1051250,1051280,1051310,1051500,1051530,1051810,1051830,1051840,1051890,1052010,1052070,1052220,1052480,1052760,1052850,1052870,1052900,1053040,1053060,1053090,1053160,1053190,1053250,1053300,1053650,1053660,1053680,1053730,1053740,1053780,1053960,1054240,1054250,1054560,1054650,1054750,1054790,1054900,1054930,1054980,1055000,1055090,1055140,1055430,1055620,1055690,1055770,1055890,1055970,1055990,1056260,1056470,1056660,1057180,1057390,1057420,1057430,1057460,1057500,1057660,1057690,1057710,1058150,1058350,1058430,1058590,1058660,1058910,1058930,1058940,1059090,1059190,1059280,1059500,1059710,1060030,1060110,1060170,1060300,1060440,1060770,1060870,1061230,1061470,1062120,1062670,1062880,1063060,1063230,1063560,1064060,1064580,1064890,1065160,1065230,1065570,1065650,1066700,1069460
gen_score,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,0.333333,0.5,1.0,0.5,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.333333,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.333333,1.0,0.0,0.0,0.333333,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.5,0.5,0.333333,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,1.0,0.0,1.0,1.0,0.0,1.0,0.2,1.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.333333,0.5,0.5,0.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.0,0.0,0.0,0.0,0.333333,0.25,0.5,0.5,0.0,0.333333,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.5,0.166667,0.333333,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.25,1.0,0.0,0.5,0.5,0.5,0.0,0.0,0.5,0.0,0.0,0.333333,0.0,0.0,0.0,0.142857,0.0,1.0,0.333333,0.0,0.0,0.2,0.0,0.2,0.0,0.333333,0.0,0.0,0.0,0.333333,0.142857,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.333333,0.0,0.25,0.166667,0.0,0.0,0.333333,0.0,0.1,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.2,0.333333,0.333333,0.333333,0.25,0.111111,0.25,0.0,0.333333,0.0,0.0,0.2,0.0,0.333333,0.333333,0.0,0.0,0.0,0.0,0.

,10,20,30,40,50,60,70,80,130,220,240,280,300,320,340,360,380,400,420,440,500,550,570,620,630,730,1002,1200,1250,1300,1500,1510,1520,1530,1600,1610,1630,1640,1670,1690,1700,1840,1900,1930,2100,2200,2210,2270,2280,2290,2300,2310,2320,2330,2340,2350,2360,2370,2390,2400,2450,2500,2520,2590,2600,2610,2620,2630,2640,2710,2720,2780,2800,2810,2820,2840,2850,2870,2900,2910,2920,2990,3010,3020,3050,3130,3170,3230,3260,3270,3330,3410,3480,3483,3490,3500,3510,3520,3530,3540,3560,3570,3580,3590,3600,3610,3620,3700,3710,3720,3730,3800,3810,3820,3830,3900,3910,3920,3960,3980,3990,4000,4100,4230,4290,4300,4420,4460,4470,4500,4520,4530,4560,4570,4580,4700,4720,4760,4770,4780,4800,4850,4870,4880,4890,4900,4920,6000,6010,6020,6030,6040,6060,6080,6090,6120,6200,6210,6220,6250,6270,6300,6310,6370,6400,6420,6510,6550,6800,6810,6830,6840,6850,6860,6870,6880,6900,6910,6920,6980,7000,7010,7020,7110,7200,7210,7220,7260,7340,7510,7520,7530,7600,7610,7620,7650,7660,7730,7760,7770,...,1043480,1043500,1043510,1043560,1043580,1043610,1043680,1043730,1043740,1043890,1044170,1044200,1044240,1044340,1044350,1044450,1044530,1044630,1044640,1044770,1044830,1044840,1044920,1044950,1045020,1045080,1045220,1045300,1045530,1045740,1045850,1045930,1046030,1046070,1046110,1046230,1046240,1046330,1046370,1046430,1046490,1046530,1046560,1046670,1046750,1046820,1047120,1047140,1047160,1047190,1047240,1047670,1047720,1047780,1047910,1047960,1048000,1048040,1048080,1048100,1048170,1048260,1048410,1048470,1048570,1048640,1048830,1048850,1048920,1048960,1049070,1049080,1049090,1049140,1049230,1049270,1049370,1049420,1049660,1049680,1049730,1049800,1049840,1049910,1049930,1049950,1050010,1050150,1050190,1050210,1050230,1050240,1050430,1050470,1050690,1050730,1050760,1050870,1051130,1051160,1051170,1051250,1051280,1051310,1051500,1051530,1051810,1051830,1051840,1051890,1052010,1052070,1052220,1052480,1052760,1052850,1052870,1052900,1053040,1053060,1053090,1053160,1053190,1053250,1053300,1053650,1053660,1053680,1053730,1053740,1053780,1053960,1054240,1054250,1054560,1054650,1054750,1054790,1054900,1054930,1054980,1055000,1055090,1055140,1055430,1055620,1055690,1055770,1055890,1055970,1055990,1056260,1056470,1056660,1057180,1057390,1057420,1057430,1057460,1057500,1057660,1057690,1057710,1058150,1058350,1058430,1058590,1058660,1058910,1058930,1058940,1059090,1059190,1059280,1059500,1059710,1060030,1060110,1060170,1060300,1060440,1060770,1060870,1061230,1061470,1062120,1062670,1062880,1063060,1063230,1063560,1064060,1064580,1064890,1065160,1065230,1065570,1065650,1066700,1069460
cat_score,1.0,1.0,0.333333,1.0,0.25,0.666667,0.5,0.25,0.0,0.0,0.333333,0.0,0.333333,0.333333,0.0,0.333333,0.0,0.0,0.0,0.333333,0.2,0.2,0.25,0.0,0.2,0.333333,0.25,0.333333,0.2,0.0,0.0,0.0,0.25,0.25,0.25,0.25,0.2,0.2,0.25,0.25,0.0,0.0,0.2,0.4,0.25,0.25,0.25,0.0,0.0,0.0,0.25,0.2,0.2,0.25,0.25,0.25,0.0,0.0,0.25,0.25,0.25,0.0,0.0,0.0,0.0,0.0,0.25,0.25,0.25,0.25,0.25,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.2,0.0,0.25,0.25,0.25,0.0,0.0,0.25,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.25,0.0,0.0,0.0,0.0,0.25,0.25,0.0,0.25,0.0,0.0,0.2,0.0,0.25,0.25,0.0,0.0,0.0,0.25,0.25,0.2,0.2,0.25,0.25,0.25,0.25,0.0,0.25,0.25,0.25,0.25,0.25,0.0,0.0,0.75,0.0,0.666667,0.5,0.0,0.5,0.5,0.0,0.5,0.142857,0.0,0.0,0.0,0.0,0.25,0.25,0.25,0.0,0.0,0.2,0.166667,0.25,0.25,0.0,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.25,0.2,0.0,0.25,0.0,0.0,0.25,0.0,0.0,0.333333,0.0,0.25,0.25,0.0,0.0,0.0,0.25,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.4,0.0,0.0,0.2,0.0,0.0,0.25,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,

In [45]:
# smoting needs to be between 0 and 1 in order to transform the linear value in a curve with slope to top left
def date_score(df, index, smooting):
    target = df.loc[index].values[0]
    df = (1-abs(df-target))**smooting
    df.columns = ['date_score']
    return df
    
date_score(date_df, 10, 0.8).T

,10,20,30,40,50,60,70,80,130,220,240,280,300,320,340,360,380,400,420,440,500,550,570,620,630,730,1002,1200,1250,1300,1500,1510,1520,1530,1600,1610,1630,1640,1670,1690,1700,1840,1900,1930,2100,2200,2210,2270,2280,2290,2300,2310,2320,2330,2340,2350,2360,2370,2390,2400,2450,2500,2520,2590,2600,2610,2620,2630,2640,2710,2720,2780,2800,2810,2820,2840,2850,2870,2900,2910,2920,2990,3010,3020,3050,3130,3170,3230,3260,3270,3330,3410,3480,3483,3490,3500,3510,3520,3530,3540,3560,3570,3580,3590,3600,3610,3620,3700,3710,3720,3730,3800,3810,3820,3830,3900,3910,3920,3960,3980,3990,4000,4100,4230,4290,4300,4420,4460,4470,4500,4520,4530,4560,4570,4580,4700,4720,4760,4770,4780,4800,4850,4870,4880,4890,4900,4920,6000,6010,6020,6030,6040,6060,6080,6090,6120,6200,6210,6220,6250,6270,6300,6310,6370,6400,6420,6510,6550,6800,6810,6830,6840,6850,6860,6870,6880,6900,6910,6920,6980,7000,7010,7020,7110,7200,7210,7220,7260,7340,7510,7520,7530,7600,7610,7620,7650,7660,7730,7760,7770,...,1043480,1043500,1043510,1043560,1043580,1043610,1043680,1043730,1043740,1043890,1044170,1044200,1044240,1044340,1044350,1044450,1044530,1044630,1044640,1044770,1044830,1044840,1044920,1044950,1045020,1045080,1045220,1045300,1045530,1045740,1045850,1045930,1046030,1046070,1046110,1046230,1046240,1046330,1046370,1046430,1046490,1046530,1046560,1046670,1046750,1046820,1047120,1047140,1047160,1047190,1047240,1047670,1047720,1047780,1047910,1047960,1048000,1048040,1048080,1048100,1048170,1048260,1048410,1048470,1048570,1048640,1048830,1048850,1048920,1048960,1049070,1049080,1049090,1049140,1049230,1049270,1049370,1049420,1049660,1049680,1049730,1049800,1049840,1049910,1049930,1049950,1050010,1050150,1050190,1050210,1050230,1050240,1050430,1050470,1050690,1050730,1050760,1050870,1051130,1051160,1051170,1051250,1051280,1051310,1051500,1051530,1051810,1051830,1051840,1051890,1052010,1052070,1052220,1052480,1052760,1052850,1052870,1052900,1053040,1053060,1053090,1053160,1053190,1053250,1053300,1053650,1053660,1053680,1053730,1053740,1053780,1053960,1054240,1054250,1054560,1054650,1054750,1054790,1054900,1054930,1054980,1055000,1055090,1055140,1055430,1055620,1055690,1055770,1055890,1055970,1055990,1056260,1056470,1056660,1057180,1057390,1057420,1057430,1057460,1057500,1057660,1057690,1057710,1058150,1058350,1058430,1058590,1058660,1058910,1058930,1058940,1059090,1059190,1059280,1059500,1059710,1060030,1060110,1060170,1060300,1060440,1060770,1060870,1061230,1061470,1062120,1062670,1062880,1063060,1063230,1063560,1064060,1064580,1064890,1065160,1065230,1065570,1065650,1066700,1069460
date_score,1.0,0.941382,0.90752,0.978676,0.963114,1.0,0.926688,0.876035,0.978676,0.848971,0.850538,0.866483,0.62536,0.850538,0.812721,0.79301,0.789713,0.736476,0.736476,0.736476,0.692394,0.651732,0.498678,0.59357,0.624571,0.53676,0.814305,0.798108,0.672749,0.792053,0.823795,0.780868,0.776917,0.698874,0.801821,0.801821,0.785986,0.785986,0.783855,0.7751,0.756979,0.541676,0.796197,0.674299,0.774138,0.74383,0.581195,0.74383,0.74383,0.74383,0.74383,0.74383,0.74383,0.74383,0.74383,0.74383,0.74383,0.74383,0.74383,0.785453,0.613034,0.792265,0.76825,0.733442,0.758269,0.775421,0.775421,0.775421,0.775421,0.719752,0.746205,0.719752,0.784388,0.784388,0.69591,0.615414,0.615414,0.483219,0.774031,0.774031,0.774031,0.768036,0.780761,0.733334,0.696789,0.653741,0.651732,0.743506,0.74102,0.734309,0.780121,0.780121,0.759989,0.739614,0.74804,0.735826,0.729537,0.71048,0.675959,0.69602,0.692284,0.684464,0.682147,0.673746,0.675959,0.66454,0.658979,0.665762,0.678171,0.668426,0.64536,0.778519,0.778519,0.778519,0.775635,0.774138,0.774138,0.824111,0.774138,0.756656,0.756656,0.770393,0.776276,0.770928,0.748363,0.772747,0.758484,0.697777,0.705778,0.758484,0.745666,0.745666,0.745666,0.743398,0.743398,0.771891,0.528658,0.741128,0.741128,0.741128,0.771142,0.619488,0.578666,0.578666,0.578666,0.768786,0.52854,0.66665,0.66665,0.658867,0.658867,0.66665,0.66665,0.66665,0.66665,0.613488,0.768036,0.768036,0.763425,0.759237,0.738532,0.765678,0.753

# Game Recommendation System Functions

In this section, I created five functions to get similar games based on features and scores.

## Function 1: score_tab

This function takes the target game index and the stored dataframes as input, and calculates the scores for each feature, such as developers, publishers, genres, categories, tags, ratings, release dates, and descriptions. It then joins these scores into a single DataFrame and returns it.

## Function 2: get_score

This function combines all the scores from the first function into a single score by assigning different weights to each feature based on their importance. These weights were determined through knowledge of video games, discussions with gamers, and testing. The function then sorts the games by their total score and returns the top 30 similar games.

The weights in this function are designed to give more importance to games released around the same time as the target game. This is based on the assumption that users looking for a specific game might be more interested in other games from the same time period.

## Function 3: get_score_new_games

This function is similar to the previous one but adjusts the weights to give more importance to the release date, specifically favoring newer games. This function was created to ensure that the recommendation system also includes new games for users to discover, regardless of the target game's release date. The other weights in this function were fine-tuned through extensive testing with known games.

## Function 4: get_results

This function combines the results from the `get_score` and `get_score_new_games` functions by selecting the top 4 similar games and 3 random games from the top 30, and then removing any common games from the similar recent games list. It then adds the top 3 new games and 2 random games from the new games list, resulting in a total of 12 recommended games, which is the same number of games Steam displays on their website. The function then shuffles the results randomly and returns them.

## Function 5: get_recommendations

This function takes a game index as input and calls all the previous functions to provide the 12 recommended games in a single line of code. The final recommendations include a mix of similar games, random selections from the top matches, and a focus on both new and older games to provide a diverse set of recommendations for the user.


In [46]:
def score_tab(target,
              scores_df = stored_tables['scores_df'], 
              dev_df = stored_tables['dev_df'],
              pub_df = stored_tables['pub_df'],
              tag_df = stored_tables['tag_df'],
              gen_df = stored_tables['gen_df'],
              cat_df = stored_tables['cat_df'],
              rating_df = stored_tables['rating_df'], 
              date_df = stored_tables['date_df'],
              des_vectors_df = stored_tables['des_vectors_df']):
    
    scores= scores_df.join([
                            get_dev_scores(dev_df, target),
                            get_pub_scores(pub_df, target),
                            single_row_jaccard_similarity(gen_df, target, 'gen_score'),
                            single_row_jaccard_similarity(cat_df, target, 'cat_score'),
                            single_row_cosine_similarity(tag_df,target, 'tag_score'),
                            rating_df,
                            date_score(date_df, target, 0.7),
                            single_row_cosine_similarity(des_vectors_df,target, 'des_score'),
                            ]).drop(target, axis=0)
    return scores


def get_score(scores, top_val=30):
    multipliers = {'dev_score': 5,
                   'pub_score': 6,
                   'gen_score': 7,
                   'cat_score': 8,
                   'tag_score': 10,
                   'weighted_rating': 35,
                   'date_score': 9,
                   'des_score': 20}
    scores = scores.mul(pd.Series(multipliers), axis=1) 

    score = pd.DataFrame(scores.sum(axis=1), columns=['SCORE'])
    score = score.sort_values(by='SCORE', ascending=False)
    
    return score.head(top_val)/100

# elevating by a number bigger then 1 the days_norm will became a ripid curve,
# decreasing the scores of values more distant to 1 
def get_score_new_games(scores, date_df = stored_tables['date_df'], top_val=30):
    scores['date_score'] = (1-date_df['days_norm'])**5
    multipliers = {'dev_score': 6,
                   'pub_score': 7,
                   'gen_score': 5,
                   'cat_score': 6,
                   'tag_score': 10,
                   'weighted_rating': 10,
                   'date_score': 50,
                   'des_score': 1}
    scores = scores.mul(pd.Series(multipliers), axis=1) 
    
    score = pd.DataFrame(scores.sum(axis=1), columns=['SCORE'])
    score = score.sort_values(by='SCORE', ascending=False)
    
    return score.head(top_val)/100


def get_results(score, score_new_games, drop_score=True, add_titles=True, title=stored_tables['title']):
    result = pd.concat([score[:4], 
                        score[4:].sample(3)], 
                        axis=0)

    score_new_games = score_new_games.drop(index=result.index, errors='ignore')

    new_games_results = pd.concat([score_new_games[:3], 
                                   score_new_games[3:].sample(2)],
                                   axis=0)
    
    result = pd.concat([result, new_games_results], axis=0).sample(frac=1)
    if add_titles:
        result = result.join(title)
    
    if drop_score:
        result = result.drop(columns=['SCORE'])
        
    return result

def get_recommendations(target, print_target_game=False, drop_score=True,  add_titles=True):
    if print_target_game:
        print('Target game:', name_df.loc[target][0])
        
    scores = score_tab(target)
    score = get_score(scores)
    score_new_games = get_score_new_games(scores)
    
    return get_results(score, score_new_games, drop_score, add_titles)

In [47]:
# Known games' indexes for test: 10, 45700, 6550, 566050, 289070
# Remove comment to the second target assigment to test on a random game 

target = 289070
# target = df.sample().index[0]

print('Target:', name_df.loc[target][0])
print('index:', target)


Target: Sid Meier’s Civilization® VI
index: 289070


In [48]:
get_recommendations(target)


,name
8930,Sid Meier's Civilization® V
1005930,Timeflow – Time and Money Simulator
200510,XCOM: Enemy Unknown
8800,Civilization IV: Beyond the Sword
4700,Total War: MEDIEVAL II – Definitive Edition
965240,Akabeth Tactics
1058590,Franchise Wars
977690,Skyworld: Kingdom Brawl
603850,Age of Civilizations II
48950,Greed Corp


### Example of each step

In [49]:
scores = score_tab(target)
scores.sample(5)

,dev_score,pub_score,gen_score,cat_score,tag_score,weighted_rating,date_score,des_score
264560,0.0,0.0,0.000000,0.5,0.000000,0.679699,0.925683,0.943874
850690,0.0,0.0,0.000000,0.5,0.000000,0.250000,0.946092,0.955254
317940,0.0,0.0,0.000000,0.5,0.026965,0.431868,0.952733,0.970279
385350,0.0,0.0,0.333333,0.5,0.320498,0.442105,0.984058,0.969294
316370,0.0,0.0,0.000000,0.5,0.000000,0.334091,0.969073,0.974595


In [50]:
s = get_score(scores)
s.head().join(name_df)


,SCORE,name
8930,0.933661,Sid Meier's Civilization® V
3900,0.872145,Sid Meier's Civilization® IV
200510,0.871319,XCOM: Enemy Unknown
8800,0.848674,Civilization IV: Beyond the Sword
3910,0.843349,Sid Meier's Civilization® III Complete


In [51]:
s_new_games = get_score_new_games(scores)
s_new_games.join(name_df).join(stored_tables['rel_date']).head()

,SCORE,name,release_date
1058590,0.700207,Franchise Wars,2019-04-15
965240,0.699047,Akabeth Tactics,2019-04-11
603850,0.695300,Age of Civilizations II,2018-11-21
798510,0.685977,SUPER DRAGON BALL HEROES WORLD MISSION,2019-04-04
1046030,0.685899,ISLANDERS,2019-04-04


In [52]:
get_results(s, s_new_games, drop_score=False, add_titles=True)

,SCORE,name
603850,0.695300,Age of Civilizations II
1058590,0.700207,Franchise Wars
3900,0.872145,Sid Meier's Civilization® IV
607050,0.656442,Wargroove
8930,0.933661,Sid Meier's Civilization® V
65980,0.782468,Sid Meier's Civilization®: Beyond Earth™
221380,0.809435,Age of Empires II HD
40970,0.787063,Stronghold Crusader HD
200510,0.871319,XCOM: Enemy Unknown
8800,0.848674,Civilization IV: Beyond the Sword


# Conclusions

In this game recommendation system, a variety of techniques ad been used to find and recommend similar games for users. 

Some of the key techniques and their advantages are:
- Jaccard similarity for comparing categorical features like genres and categories, which is effective in measuring similarity between sets of items.
- Cosine similarity for comparing continuous features like tags and description vectors, which can capture the similarity between items even if they are not identical.
- Weighted scores for combining various features based on their importance, which provides a comprehensive similarity score that takes multiple aspects of the games into account.

This recommendations system acheived comparable results to the ones suggested by Steam on their website and some suggested games are shared. The system's running time is approximately 1.45 seconds when the precomputed tables are provided.

However, there are some limitations, such as time constraints, limited resources, and access to more data that could potentially improve the recommendations.

Possible expansions for this recommendation system include:

- Evaluating user comments on games, although this may be biased due to the tendency of users to write short negative reviews and the potential for sentiment analysis to be misleading for certain genres like horror or gore games.
- Recommending games that are similar to the ones in a registered user's library or most-played list, which would provide a more personalized set of recommendations.
- Recommending games that are similar to those in the libraries of other users who have the target game in their libraries, based on playtime and other data that we did not have access to.
- ncorporating Steam users' libraries analysis to identify clusters of users with similar preferences and recommend games that are popular within these clusters.

By addressing these limitations and implementing the suggested expansions, it would be possible to further improve the accuracy and effectiveness of the game recommendation system, providing users with an even better experience when discovering new games to play.
